# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [ ]:
# Use as many python cells as you wish to write your code

import pandas as pd


def merge_all_data(health_data_path, supplement_usage_path, experiments_data_path, user_profiles_data_path):
    #     loading all the data
    health_data = pd.read_csv(health_data_path)
    supplement_usage_data = pd.read_csv(supplement_usage_path)
    experiments_data = pd.read_csv(experiments_data_path)
    user_profiles_data = pd.read_csv(user_profiles_data_path)
    
    #     merging supplement_usage and experiments
    supplement_usage_experiments = pd.merge(supplement_usage_data, experiments_data, on='experiment_id', how='left')
    profiles_health_data = pd.merge(health_data, user_profiles_data, on='user_id', how='left')
    
    #     merging all datasets
    all_dataset = pd.merge(profiles_health_data, supplement_usage_experiments, on=['user_id', 'date'], how='left')
    
    #     cleaning the combined_dataset: convert mg to grams
    all_dataset['dosage_grams'] = all_dataset['dosage'] / 1000
    all_dataset.drop(columns=['dosage', 'dosage_unit'], inplace=True)
    
    # Categorize age into age groups
    bins = [0, 17, 25, 35, 45, 55, 65, float('inf')]
    labels = ['Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65']
    all_dataset['user_age_group'] = pd.cut(all_dataset['age'], bins=bins, labels=labels, right=False)
    
    # Add 'Unknown' to the categories before filling missing values
    all_dataset['user_age_group'] = all_dataset['user_age_group'].cat.add_categories('Unknown')
    all_dataset['user_age_group'].fillna('Unknown', inplace=True)
    
    #     cleaning the combined_dataset: handle missing values
    all_dataset.rename(columns={'name': 'experiment_name'}, inplace=True)
    all_dataset['experiment_name'].fillna("No Experiment", inplace=True)
    all_dataset['supplement_name'].fillna("No intake", inplace=True)
    
    # Fill 'is_placebo' only where 'supplement_name' is not 'No intake'
    # all_dataset.loc[all_dataset['supplement_name'] != 'No intake', 'is_placebo'] = all_dataset.loc[all_dataset['supplement_name'] != 'No intake', 'is_placebo'].fillna(False)
    
    # Select the final columns in the specified order
    final_df = all_dataset[['user_id', 'date', 'email', 'user_age_group', 'experiment_name', 'supplement_name', 'dosage_grams', 'is_placebo',
                         'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level']]
    
    
    return final_df

In [1]:
import pandas as pd

profile_data = pd.read_csv("user_profiles.csv")
profile_data.head()

,user_id,email,age
0,c6ae338a-9f95-481c-a88d-24a58bc8fc71,hi_1@example.com,68.0
1,5346f1dc-30f7-4e3a-9d35-eec6cb8835fa,hi_2@myemail.com,64.0
2,5541289d-fa9f-4aef-9504-1e11d940efc5,hello_3@example.com,40.0
3,f8461502-54b8-4388-ae0d-9095cfc0426f,hi4@myemail.com,33.0
4,9e4781d1-111b-4318-907f-63c58da6cd89,user_5@myemail.com,48.0


In [2]:
health_data = pd.read_csv("user_health_data.csv")
health_data.head()

,user_id,date,average_heart_rate,average_glucose,sleep_hours,activity_level
0,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-01-31,93.055612,70.089910,8.8h,1
1,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-02-28,88.059964,78.411148,8.0H,3
2,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,78.373746,107.418818,11.9h,1
3,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-04-30,62.204061,117.259092,5.1h,1
4,5346f1dc-30f7-4e3a-9d35-eec6cb8835fa,2018-01-31,77.075789,99.221380,9.6h,2


In [3]:
supplement_usage = pd.read_csv("supplement_usage.csv")
supplement_usage.head()

,user_id,date,supplement_name,dosage,dosage_unit,is_placebo,experiment_id
0,38bbb850-2b41-47a2-b287-833317055c1a,2018-04-30,Vitamin C,380.742204,mg,False,14fb6430-c228-407c-ba66-a35588b98276
1,73420928-0991-41c2-9e31-930b18b61f95,2018-03-31,Magnesium,115.521810,mg,True,d655666f-725d-4312-96b4-df25045dd08e
2,1e9c58f2-86da-4025-88cb-b4d8dc8c889d,2018-01-31,Vitamin C,482.849678,mg,True,d655666f-725d-4312-96b4-df25045dd08e
3,59d3b4eb-5ad8-4d95-a4b2-9c4dfb9d3e9c,2018-01-31,Vitamin C,118.511952,mg,False,85d2741f-7964-49d4-bc1d-62f0a50d8a59
4,a89c9636-6e24-40f1-a2eb-c9bcec783f0f,2018-03-31,Vitamin C,452.501273,mg,False,ce7f2178-4f42-4511-ab41-06b8cf717f7c


In [ ]:
import pandas as pd
import numpy as np

def merge_all_data(user_health_file, supplement_usage_file, experiments_file, user_profiles_file):
    user_profiles_df = pd.read_csv('user_profiles.csv')
    user_health_data_df = pd.read_csv('user_health_data.csv')
    supplement_usage_df = pd.read_csv('supplement_usage.csv')
    experiments_df = pd.read_csv('experiments.csv')

    bins = [0, 17, 25, 35, 45, 55, 65, np.inf]
    labels = ['Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65']
    user_profiles_df['user_age_group'] = pd.cut(user_profiles_df['age'], bins=bins, labels=labels, right=False)
    user_profiles_df['user_age_group'] = user_profiles_df['user_age_group'].astype('category')
    user_profiles_df['user_age_group'] = user_profiles_df['user_age_group'].cat.add_categories('Unknown')
    user_profiles_df['user_age_group'].fillna("Unknown", inplace=True)
    user_profiles_df.drop(columns=['age'], inplace=True)

    user_health_data_df['activity_level'] = ((user_health_data_df['activity_level'] - 1) / 3) * 100
    user_health_data_df['sleep_hours'] = user_health_data_df['sleep_hours'].str.replace('[hH]', '').astype(float)

    supplement_usage_df['is_placebo'].astype(str)
    supplement_usage_df['dosage_grams'] = supplement_usage_df['dosage'] / 1000
    supplement_usage_df.drop(columns=['dosage', 'dosage_unit'], inplace=True)

    merged_df = pd.merge(user_profiles_df, user_health_data_df, on='user_id', how='left')
    # merged_df = pd.merge(merged_df, supplement_usage_df, left_on=['user_id','date'], right_on=['user_id','date'], how='left')
    merged_df = pd.merge(merged_df, supplement_usage_df, on=['user_id', 'date'], how='left')
    merged_df = pd.merge(merged_df, experiments_df, on='experiment_id', how='left')

    merged_df['supplement_name'].fillna('No intake', inplace=True)

    merged_df.drop(columns=['experiment_id', 'description'], inplace=True)

    merged_df.rename(columns={'name': 'experiment_name'}, inplace=True)

    merged_df['date'] = pd.to_datetime(merged_df['date'], format='%Y-%m-%d').dt.date

    new_order = ['user_id', 'date', 'email', 'user_age_group', 'experiment_name', 'supplement_name', 'dosage_grams',
                 'is_placebo', 'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level']

    merged_df[new_order]


    print(merged_df.head())
    print(merged_df.info())

    return merged_df

merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')